# Введение в обработку текста на естественном языке

Материалы:
* Макрушин С.В. Лекция 9: Введение в обработку текста на естественном языке\
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

## Задачи для совместного разбора

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pymorphy2

1. Считайте слова из файла `litw-win.txt` и запишите их в список `words`. В заданном предложении исправьте все опечатки, заменив слова с опечатками на ближайшие (в смысле расстояния Левенштейна) к ним слова из списка `words`. Считайте, что в слове есть опечатка, если данное слово не содержится в списке `words`. 

In [2]:
text = '''с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'''

In [24]:
import re

words = []

with open('litw-win.txt') as f:
    lines = [l for l in f]

pattern = re.compile('[а-яА-ЯёЁ]*')

for line in lines:
    words.append([w for w in pattern.findall(line) if w != ''][0])s

['и',
 'в',
 'я',
 'с',
 'а',
 'к',
 'у',
 'о',
 'н',
 'п',
 'ж',
 'б',
 'т',
 'д',
 'м',
 'ч',
 'з',
 'г',
 'е',
 'р',
 'э',
 'л',
 'х',
 'ш',
 'ф',
 'ц',
 'щ',
 'й',
 'ю',
 'ы',
 'ъ',
 'ь',
 'не',
 'на',
 'он',
 'то',
 'но',
 'же',
 'вы',
 'по',
 'да',
 'за',
 'бы',
 'ты',
 'от',
 'из',
 'ее',
 'до',
 'ну',
 'ни',
 'ли',
 'уж',
 'во',
 'их',
 'мы',
 'со',
 'ей',
 'об',
 'ко',
 'ах',
 'им',
 'ка',
 'та',
 'пр',
 'те',
 'чт',
 'ту',
 'го',
 'де',
 'вс',
 'эт',
 'мо',
 'ра',
 'ст',
 'ха',
 'хе',
 'ум',
 'се',
 'эх',
 'гм',
 'ме',
 'св',
 'ль',
 'ег',
 'пе',
 'ва',
 'са',
 'ве',
 'ох',
 'бо',
 'хо',
 'че',
 'сл',
 'од',
 'бе',
 'мн',
 'ай',
 'ею',
 'ос',
 'ск',
 'ви',
 'ми',
 'ма',
 'сп',
 'ба',
 'ес',
 'бу',
 'гл',
 'що',
 'эй',
 'ем',
 'кр',
 'см',
 'др',
 'лю',
 'па',
 'ро',
 'зн',
 'ле',
 'як',
 'тр',
 'жи',
 'ел',
 'ус',
 'фу',
 'дв',
 'ру',
 'ал',
 'ой',
 'си',
 'ду',
 'вз',
 'ещ',
 'хи',
 'чи',
 'вд',
 'ил',
 'ух',
 'ча',
 'гр',
 'пи',
 'бл',
 'ре',
 'су',
 'ге',
 'ку',
 'оп',
 'ж

In [34]:
from razdel import tokenize

txt = list(tokenize(text))
txt = [txt.text for txt in txt]
op = [w for w in txt if w not in words]
op

['велечайшим', 'Кутузов', 'уменьшевшейся']

In [40]:
from nltk.metrics import edit_distance
from tqdm import tqdm

k = []

for w in tqdm(words):
    for op0 in op:
        dist = edit_distance(op0, w)
        k.append((op0, w, dist))

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 162166/162166 [00:22<00:00, 7285.70it/s]


In [46]:
import pandas as pd

dist_df = pd.DataFrame(k, columns=['op_words', 'dict_word', 'dist'])
dist_df = dist_df.sort_values(['op_words', 'dist'])
dist_df = dist_df.drop_duplicates(subset=['op_words'])
dist_df = dist_df.drop('dist', axis='columns')
dist_df

,op_words,dict_word
120376,Кутузов,кутузов
331605,велечайшим,величайшим
486497,уменьшевшейся,уменьшившейся


In [61]:
w1 = text.split(' ')
w1 = [dist_df['dict_word'][dist_df['op_words'] == w].values[0] if w in op else w for w in w1]
new_text = ' '.join(w1)
new_text

'с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий'

In [62]:
print(new_text)
print(text)

с величайшим усилием выбравшись из потока убегающих людей кутузов со свитой уменьшившейся вдвое поехал на звуки выстрелов русских орудий
с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий


'с велечайшим усилием выбравшись из потока убегающих людей Кутузов со свитой уменьшевшейся вдвое поехал на звуки выстрелов русских орудий'

2. Разбейте текст из формулировки задания 1 на слова; проведите стемминг и лемматизацию слов.

3. Преобразуйте предложения из формулировки задания 1 в векторы при помощи `CountVectorizer`.

## Лабораторная работа 9

### Расстояние редактирования

1.1 Загрузите предобработанные описания рецептов из файла `preprocessed_descriptions.csv`. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). 

1.2 Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние редактирования.

1.3 Напишите функцию, которая для заданного слова `word` возвращает `k` ближайших к нему слов из списка `words` (близость слов измеряется с помощью расстояния Левенштейна)

### Стемминг, лемматизация

2.1 На основе результатов 1.1 создайте `pd.DataFrame` со столбцами: 
    * word
    * stemmed_word 
    * normalized_word 

Столбец `word` укажите в качестве индекса. 

Для стемминга воспользуйтесь `SnowballStemmer`, для нормализации слов - `WordNetLemmatizer`. Сравните результаты стемминга и лемматизации.

2.2. Удалите стоп-слова из описаний рецептов. Какую долю об общего количества слов составляли стоп-слова? Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

### Векторное представление текста

3.1 Выберите случайным образом 5 рецептов из набора данных. Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`

3.2 Вычислите близость между каждой парой рецептов, выбранных в задании 3.1, используя косинусное расстояние (`scipy.spatial.distance.cosine`) Результаты оформите в виде таблицы `pd.DataFrame`. В качестве названий строк и столбцов используйте названия рецептов.

3.3 Какие рецепты являются наиболее похожими? Прокомментируйте результат (словами).